In [1]:
# Using LinearSVC model after fine-Tuning
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load the dataset
file_path = r'C:\Users\User\Desktop\NLP Assignment 03 by 031\Tweets.csv'
data = pd.read_csv(file_path)

# Preprocessing
data['text'] = data['text'].str.lower()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_words)

data['processed_text'] = data['text'].apply(preprocess_text)

# Split data into train and test sets
X = data['processed_text']
y = data['airline_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

param_grid = {
    'C': [0.1, 1, 10],
    'max_iter': [3000, 4000, 5000],
    'loss': ['hinge', 'squared_hinge']
}

grid_search = GridSearchCV(LinearSVC(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)
best_svc = grid_search.best_estimator_


predictions = best_svc.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, predictions)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1_score:.4f}")
print("Best Parameters:", grid_search.best_params_)


Accuracy: 0.7971
Precision: 0.7864
Recall: 0.7971
F1-score: 0.7873
Best Parameters: {'C': 1, 'loss': 'hinge', 'max_iter': 3000}


C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [3]:
# Using RandomForestClassifier model after Hyperparameter-Tuning.
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load the dataset
file_path = r'C:\Users\User\Desktop\NLP Assignment 03 by 031\Tweets.csv'
data = pd.read_csv(file_path)

# Preprocessing
data['text'] = data['text'].str.lower()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_words)

data['processed_text'] = data['text'].apply(preprocess_text)

# Split data into train and test sets
X = data['processed_text']
y = data['airline_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)
best_rf = grid_search.best_estimator_

# Predict sentiment on the test set
predictions = best_rf.predict(X_test_tfidf)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Best Parameters:", grid_search.best_params_)


Accuracy: 0.7715
Classification Report:
              precision    recall  f1-score   support

    negative       0.79      0.94      0.86      1889
     neutral       0.63      0.40      0.49       580
    positive       0.77      0.55      0.64       459

    accuracy                           0.77      2928
   macro avg       0.73      0.63      0.66      2928
weighted avg       0.76      0.77      0.75      2928

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}


In [5]:
# Now cleaning our Tweets.csv file
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load the dataset
file_path = r'C:\Users\User\Desktop\NLP Assignment 03 by 031\Tweets.csv'
data = pd.read_csv(file_path)

# Lowercasing text
data['text'] = data['text'].str.lower()

# Function for text cleaning
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove mentions and special characters
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stopwords and lemmatize words
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # Join tokens back to text
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

# Apply text cleaning to 'text' column
data['cleaned_text'] = data['text'].apply(clean_text)

# Handling missing values if any
data.dropna(subset=['cleaned_text'], inplace=True)

# Displaying cleaned data
cleaned_data = data[['cleaned_text', 'airline_sentiment']]
print(cleaned_data.head())

# Save cleaned data to a new CSV file
cleaned_data.to_csv('cleaned_data.csv', index=False)


                                        cleaned_text airline_sentiment
0                        virginamerica dhepburn said           neutral
1  virginamerica plus added commercial experience...          positive
2             today must mean need take another trip           neutral
3  really aggressive blast obnoxious entertainmen...          negative
4                               really big bad thing          negative


In [9]:
# Using SVM Model after hyperparameter Tuning.
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Load the dataset
file_path = 'Tweets.csv'
data = pd.read_csv(file_path)

# Preprocessing
data['text'] = data['text'].str.lower()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        text = re.sub(r'http\S+', '', text)  # Remove URLs
        text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove @mentions
        words = word_tokenize(text)
        filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
        return ' '.join(filtered_words)
    else:
        return ''  # Return empty string for non-string values

data['processed_text'] = data['text'].apply(preprocess_text)

# Remove rows with empty processed_text
data = data[data['processed_text'].str.len() > 0]

# Split data into train and test sets
X = data['processed_text']
y = data['airline_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize and train the SVC model with grid search for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

best_svm = grid_search.best_estimator_

# Predict sentiment on the test set
predictions = best_svm.predict(X_test_tfidf)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print(f"Accuracy with SVM: {accuracy:.4f}")
print(f"Precision with SVM: {precision:.4f}")
print(f"Recall with SVM: {recall:.4f}")
print(f"F1 Score with SVM: {f1:.4f}")
print("Best Parameters for SVM:", grid_search.best_params_)


Accuracy with SVM: 0.7838
Precision with SVM: 0.7743
Recall with SVM: 0.7838
F1 Score with SVM: 0.7755
Best Parameters for SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Load the dataset
file_path = 'Tweets.csv'
data = pd.read_csv(file_path)

# Preprocessing
data['text'] = data['text'].str.lower()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        text = re.sub(r'http\S+', '', text)  # Remove URLs
        text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove @mentions
        words = word_tokenize(text)
        filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
        return ' '.join(filtered_words)
    else:
        return ''  # Return empty string for non-string values

data['processed_text'] = data['text'].apply(preprocess_text)

# Remove rows with empty processed_text
data = data[data['processed_text'].str.len() > 0]

# Split data into train and test sets
X = data['processed_text']
y = data['airline_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Fine-tuning the Multinomial Naive Bayes Classifier model using GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False]
}

grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)
best_nb = grid_search.best_estimator_

# Predict sentiment on the test set
predictions = best_nb.predict(X_test_tfidf)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print(f"Accuracy with Multinomial Naive Bayes: {accuracy:.4f}")
print(f"Precision with Multinomial Naive Bayes: {precision:.4f}")
print(f"Recall with Multinomial Naive Bayes: {recall:.4f}")
print(f"F1 Score with Multinomial Naive Bayes: {f1:.4f}")

print("Best Parameters for Multinomial Naive Bayes:", grid_search.best_params_)


Accuracy with Multinomial Naive Bayes: 0.7622
Precision with Multinomial Naive Bayes: 0.7532
Recall with Multinomial Naive Bayes: 0.7622
F1 Score with Multinomial Naive Bayes: 0.7423
Best Parameters for Multinomial Naive Bayes: {'alpha': 0.1, 'fit_prior': True}
